# Tarea 3: utilizar Amazon Bedrock para responder a preguntas

En este cuaderno, aprenderá a utilizar el modelo Bedrock Titan para proporcionar respuestas informativas a las consultas; para ello, deberá enviar la solicitud con todo el contexto pertinente al modelo y esperar la respuesta, lo que aborda el desafío de que el modelo devuelva respuestas fácticas a las preguntas sin necesidad de preparar e indexar los documentos con anticipación.

Este cuaderno simula lo que haría la **generación aumentada por recuperación (RAG)**, pero en realidad no utiliza RAG. Este enfoque funciona con documentos cortos o aplicaciones únicas. Es posible que no responda a preguntas a nivel empresarial, ya que allí los documentos empresariales grandes no caben en la petición que se envía al modelo.

La tarea de respuesta a preguntas **(Question Answering, QA)** es importante, ya que implica la extracción de respuestas en relación con consultas objetivas planteadas en lenguaje natural. En general, el sistema de QA procesa una consulta frente a una base de conocimiento que contiene datos estructurados o no estructurados y genera una respuesta con información precisa. Garantizar una alta exactitud es clave para desarrollar un sistema de respuesta a preguntas que sea útil, fiable y de confianza, en especial en lo que respecta a los casos prácticos empresariales.


## Situación

Intenta modelar una situación en AnyCompany y pide al modelo de respuesta a preguntas que proporcione información acerca del cambio de neumáticos de un modelo de vehículo específico que fabrican. Primero, consulta el modelo con un enfoque "sin entrenamiento previo" para ver si puede proporcionar respuestas adecuadas en función de sus datos de entrenamiento únicamente.

Sin embargo, se da cuenta de que el modelo parece estar "alucinando" con respuestas más genéricas, como se demuestra cuando prueba con un modelo de vehículo falso y obtiene respuestas similares. Esto implica la necesidad de aumentar el entrenamiento del modelo con los manuales de vehículos reales de Empresa de Ejemplo para dar detalles sobre los neumáticos de cada modelo.

En este laboratorio, simulará un enfoque de "generación aumentada por recuperación" (RAG) sin datos externos. Proporcionará un extracto detallado del manual en el que se explica cómo cambiar los neumáticos del vehículo Modelo Z de AnyCompany. Comprueba si el modelo ahora puede dar una respuesta precisa y personalizada que aproveche este contenido de ejemplo contextualizado.

## Tarea 3.1: configuración del entorno

En esta tarea, establecerá el entorno.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tarea 3.2: preguntas y respuestas con el conocimiento del modelo
En esta sección, se intenta utilizar un modelo que proporciona el servicio Bedrock para responder a preguntas a partir del conocimiento obtenido durante la fase de entrenamiento.

En esta tarea, utilizará el método invoke_model() del cliente Amazon Bedrock. Los parámetros obligatorios para utilizar este método son modelId, que representa el ARN del modelo de Amazon Bedrock, y body, que es la petición de la tarea.

La petición body cambia en función del proveedor del modelo fundacional que seleccione. Esto se analizará en detalle a continuación.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Se intenta utilizar los modelos que proporciona el servicio Bedrock para responder a preguntas a partir del conocimiento obtenido durante la fase de entrenamiento.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## Tarea 3.3: invocar el modelo en el cuerpo JSON para generar la respuesta

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


El modelo ofrece una respuesta en la que se detalla el proceso de cambio de un neumático desinflado en un automóvil, pero esa misma explicación sería válida para cualquier automóvil. Desafortunadamente, esta no es la respuesta correcta para un AC8 de AnyCompany, que no cuenta con un neumático de repuesto. Esto ocurre porque el modelo se entrenó con datos que contienen instrucciones acerca de cómo cambiar neumáticos de automóviles.

Otro ejemplo de este problema se observa cuando se intenta hacer la misma pregunta en relación con una marca y un modelo falsos de automóvil, como un Amazon Tirana.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

En función de la pregunta de la petición, el modelo no puede ofrecer una respuesta realista.

Para solucionar este problema y hacer que el modelo proporcione respuestas en función de las instrucciones específicas válidas para el modelo de automóvil, puede aumentar los conocimientos del modelo al instante. Para ello, proporcione una base de conocimientos adicional como parte de la petición.

Observe cómo puede aprovechar esto para mejorar la aplicación.

Supongamos que lo siguiente es un extracto del manual del AC8 de AnyCompany (en realidad, no es el manual verdadero, pero trátelo como tal). Además, este documento es convenientemente corto como para ajustarse a la ventana contextual de Titan Large.

```plain
Neumáticos y su presión:

Los neumáticos son de goma negra y se montan sobre las ruedas de su vehículo. Ofrecen el agarre necesario para conducir, girar y frenar. La presión y el desgaste de los neumáticos son dos factores importantes que se deben considerar, ya que pueden afectar el rendimiento y el manejo del automóvil.

En dónde buscar la presión recomendada para neumáticos:

Puede buscar las especificaciones sobre presión recomendada para neumáticos en la etiqueta de inflado sobre el pilar B del lado del conductor del vehículo. Como alternativa, puede consultar el manual del vehículo para obtener esta información. La presión recomendada para neumáticos puede variar en función de la velocidad y la cantidad de ocupantes o de la carga máxima del vehículo.

Volver a inflar los neumáticos:

Es importante verificar la presión de los neumáticos cuando estos estén fríos. Esto significa que el vehículo debe descansar durante al menos tres horas a fin de asegurarse de que los neumáticos estén a la misma temperatura que el ambiente.

Para volver a inflar los neumáticos, realice lo siguiente:

    Verifique la presión recomendada para neumáticos del vehículo.
    Siga las instrucciones proporcionadas en la bomba de aire e infle los neumáticos a la presión correcta.
    En la pantalla central del vehículo, abra la aplicación "Car status".
    Vaya a la pestaña "Tire pressure" (Presión de los neumáticos).
    Presione la opción "Calibrate pressure" (Calibrar presión) y confirme la acción.
    Conduzca el automóvil durante unos minutos a una velocidad por encima de 30 km/h para calibrar la presión de los neumáticos.

Nota: En algunos casos, es posible que sea necesario conducir durante más de 15 minutos para borrar símbolos de advertencia o mensajes relacionados con la presión de los neumáticos. Si las advertencias persisten, deje que los neumáticos se enfríen y repita los pasos anteriores.

Neumático desinflado:

Si uno de sus neumáticos se desinfla mientras conduce, puede sellar temporalmente la pinchadura y volver a inflar el neumático con el kit de reparación de neumáticos. Este kit, en general, se almacena por debajo del revestimiento del área de equipaje en el vehículo.

Instrucciones para usar el kit de reparación de neumáticos:

    Abra la puerta trasera o la caja del vehículo.
    Levante el revestimiento del área de equipaje para acceder al kit de reparación de neumáticos.
    Siga las instrucciones que acompañan el kit para sellar la pinchadura del neumático.
    Después de usar el kit, asegúrese de colocarlo de nuevo en el lugar de origen.
    Póngase en contacto con Rivesla o con el servicio apropiado para obtener asistencia en cuanto a cómo desechar y reemplazar la botella de sellador usada.

Tenga en cuenta que el kit de reparación de neumáticos es una solución temporal y que se diseñó para que pueda conducir durante un máximo de 10 minutos u 8 km (lo que sea que ocurra primero) a una velocidad máxima de 80 km/h. Se recomienda cambiar el neumático pinchado o solicitar su reparación por parte de un profesional lo antes posible.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Ahora, pase el extracto completo al modelo junto con la pregunta.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### Tarea 3.4: invocar el modelo mediante boto3 para generar la respuesta

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Dado que el modelo demora un poco en comprender el contexto y generar una respuesta pertinente, la experiencia del usuario podría ser deficiente, ya que debe esperar una respuesta durante algunos segundos.

Bedrock también admite la capacidad de streaming a partir de la cual el servicio genera la salida a medida que el modelo genera los tokens. Este es un ejemplo de cómo puede implementarlo.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

La respuesta proporciona instrucciones resumidas y paso a paso sobre cómo cambiar los neumáticos. 

Aprendió cómo puede aprovechar la generación aumentada por recuperación (RAG) o el proceso de aumento para generar una respuesta seleccionada adaptada al contexto específico y a la información proporcionada.

### Pruébelo usted mismo
- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con la **Tarea 4**.